In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

import tqdm
import zipfile
import torch
import pandas as pd


In [2]:
fhlo = pd.read_csv("../../input_data/fully_hand_labelled_output.csv")

In [3]:
translated = pd.read_csv("translated_dataset_df.csv")

In [4]:
fhlo

,Unnamed: 0.1,Unnamed: 0,label_id,text,Roken,Alcohol,Drugs
0,0,0,1.0,Beloop: \tPatiÃ«nte heeft 10 minuten van te vo...,Niets gevonden,Niets gevonden,Niets gevonden
1,1,1,1.0,Beloop: \tG5P4 36 wkGrav 1e lijnALL geen Hb Pa...,Niets gevonden,Niets gevonden,Niets gevonden
2,2,2,1.0,Conclusie: \tNormale nacontrole. Kijkt goed te...,Niets gevonden,Niets gevonden,Niets gevonden
3,3,3,1.0,Beleid: \tAlgemeen: Dagopname voor 3x PC a 2....,Niets gevonden,Niets gevonden,Niets gevonden
4,4,4,1.0,Anamnese: \t34W4D,Niets gevonden,Niets gevonden,Niets gevonden
...,...,...,...,...,...,...,...
4695,4695,95,2.0,Beloop: \tCONSULTENKAMERGezien door co-ass Y. ...,Geen gebruiker,Geen gebruiker,Geen gebruiker
4696,4696,96,2.0,Reden van komst / Verwijzing: \tReden verwijzi...,Geen gebruiker,Geen gebruiker,Geen gebruiker
4697,4697,97,2.0,"Beloop: \tMR CPG7P4M4 // MI: 4x sectio ia, 1x...",Geen gebruiker,Geen gebruiker,Geen gebruiker
4698,4698,98,2.0,Reden van komst / Verwijzing: \tReden van koms...,Geen gebruiker,Geen gebruiker,Geen gebruiker


In [5]:
translated

,Unnamed: 0,doc_id,segment_id,text,translated
0,0,0,1,Beloop: .PatiÃ«nte heeft 10 minuten van te vor...,"Course: .PatiÃ""nte has called 10 minutes in ad..."
1,1,1,1,Beloop: .G5P4 36 wkGrav 1e lijnALL geen Hb Pat...,Course: .G5P4 36 wkGrav 1st lineALL no Hb Path...
2,2,2,1,Conclusie: .Normale nacontrole. Kijkt goed ter...,Conclusion: .Normal follow-up. Looks good back...
3,3,3,1,Beleid: .Algemeen: Dagopname voor 3x PC a 2.5...,Policy: .General: Daily intake for 3x PC a 2.5...
4,4,4,1,Anamnese: .34W4D,Anamnese: .34W4D
...,...,...,...,...,...
17069,17069,4699,5,"Hoofd/hals: vochtige slijmvliezen, CVD niet ve...","Head/neck: moist mucous membranes, CVD not inc..."
17070,17070,4699,6,"Sinusritme 77/min, IMHA, normale geleidingstij...","Sinus rhythm 77/min, IMHA, normal conduction t..."
17071,17071,4699,7,"Geen wandbewegingsstoornissen. Slanke VC, col...","No wall motion disorders. Slim VC, collabs spo..."
17072,17072,4699,8,Near-collaps bij 1. .3. Pre-existente microcy...,Near-collapse at 1. 3. Pre-existent microcytic...


In [6]:
test = translated.groupby(['doc_id'])['translated'].transform(lambda x: ''.join(x))


In [8]:
test.drop_duplicates()

0        Course: .PatiÃ"nte has called 10 minutes in ad...
1        Course: .G5P4 36 wkGrav 1st lineALL no Hb Path...
2        Conclusion: .Normal follow-up. Looks good back...
3        Policy: .General: Daily intake for 3x PC a 2.5...
4                                         Anamnese: .34W4D
                               ...                        
17049    Course: .CONSULTENKAMERSeen by co-ass Y. Tinga...
17053    Reason of arrival / Reference: .Read reference...
17056    Course: .MR CPG7P4M4 // MI: 4x sectio ia, 1x o...
17060    Reason of arrival / Reference: . Reason of arr...
17064    Reason of arrival / Reference: .Reds of arriva...
Name: translated, Length: 4700, dtype: object

In [9]:
test.drop_duplicates().head(10)

0    Course: .PatiÃ"nte has called 10 minutes in ad...
1    Course: .G5P4 36 wkGrav 1st lineALL no Hb Path...
2    Conclusion: .Normal follow-up. Looks good back...
3    Policy: .General: Daily intake for 3x PC a 2.5...
4                                     Anamnese: .34W4D
5    Conclusion: .Mw called, said that things are g...
6    Course: .peripartum typed Lewis A positive. in...
7    CABG on 4 May. Pneumothorax, atelectase, infil...
8    Conclusion: .Day after procedure, CRT-DWond lo...
9    Conclusion: .On HCK after elective exchangeSet...
Name: translated, dtype: object

In [10]:
test.drop_duplicates().iloc[59]

'Anamnese: . Abbreviated consultation: Focus points from pre-operative screening:..1. Relevant history.-2018 Implantation Biotronik (ICD) Ilivia 7 ProMRI DR-T DF4 ivm secondary prevention after OHCA obv VF eci.2018 Relapse pulmonary embolism.2018 Torticollis...Anamnese: .Indication Implantation of subcutaneous automatic d defibrillator (AICD)General historyProfession/work: Manager ICT of insurance company directing and organizing/leading. Meanwhile stopped.Allergyes unknownMedication, see EPDBloedverthunnersUnder treatment in neurologist, LLZ, TorticollisEarlier operated, ICD/ICU (no ok but sedated for the OHCA. Anesthesia: other, has had ruffle with a test for icdIntoxicationsThe patient smoked since 1998, cigarettes and has stopped smoking since 2018. Number of packyearrs: 0.Notesmbt smoking: Dhr had stopped smoking just before the heart attack 4 years ago.The patient drinks alcohol (beer), less than 2 units per day.The patient does not use drugs. Tractus CirculatoriusMET score 6, dy

In [11]:
full_translated_texts = test.drop_duplicates().to_frame().reset_index()

In [12]:
full_translated_texts

,index,translated
0,0,"Course: .PatiÃ""nte has called 10 minutes in ad..."
1,1,Course: .G5P4 36 wkGrav 1st lineALL no Hb Path...
2,2,Conclusion: .Normal follow-up. Looks good back...
3,3,Policy: .General: Daily intake for 3x PC a 2.5...
4,4,Anamnese: .34W4D
...,...,...
4695,17049,Course: .CONSULTENKAMERSeen by co-ass Y. Tinga...
4696,17053,Reason of arrival / Reference: .Read reference...
4697,17056,"Course: .MR CPG7P4M4 // MI: 4x sectio ia, 1x o..."
4698,17060,Reason of arrival / Reference: . Reason of arr...


In [13]:
fhlo_translated = fhlo.merge(full_translated_texts[['translated']], left_index=True, right_index=True)

In [14]:
fhlo_translated

,Unnamed: 0.1,Unnamed: 0,label_id,text,Roken,Alcohol,Drugs,translated
0,0,0,1.0,Beloop: \tPatiÃ«nte heeft 10 minuten van te vo...,Niets gevonden,Niets gevonden,Niets gevonden,"Course: .PatiÃ""nte has called 10 minutes in ad..."
1,1,1,1.0,Beloop: \tG5P4 36 wkGrav 1e lijnALL geen Hb Pa...,Niets gevonden,Niets gevonden,Niets gevonden,Course: .G5P4 36 wkGrav 1st lineALL no Hb Path...
2,2,2,1.0,Conclusie: \tNormale nacontrole. Kijkt goed te...,Niets gevonden,Niets gevonden,Niets gevonden,Conclusion: .Normal follow-up. Looks good back...
3,3,3,1.0,Beleid: \tAlgemeen: Dagopname voor 3x PC a 2....,Niets gevonden,Niets gevonden,Niets gevonden,Policy: .General: Daily intake for 3x PC a 2.5...
4,4,4,1.0,Anamnese: \t34W4D,Niets gevonden,Niets gevonden,Niets gevonden,Anamnese: .34W4D
...,...,...,...,...,...,...,...,...
4695,4695,95,2.0,Beloop: \tCONSULTENKAMERGezien door co-ass Y. ...,Geen gebruiker,Geen gebruiker,Geen gebruiker,Course: .CONSULTENKAMERSeen by co-ass Y. Tinga...
4696,4696,96,2.0,Reden van komst / Verwijzing: \tReden verwijzi...,Geen gebruiker,Geen gebruiker,Geen gebruiker,Reason of arrival / Reference: .Read reference...
4697,4697,97,2.0,"Beloop: \tMR CPG7P4M4 // MI: 4x sectio ia, 1x...",Geen gebruiker,Geen gebruiker,Geen gebruiker,"Course: .MR CPG7P4M4 // MI: 4x sectio ia, 1x o..."
4698,4698,98,2.0,Reden van komst / Verwijzing: \tReden van koms...,Geen gebruiker,Geen gebruiker,Geen gebruiker,Reason of arrival / Reference: . Reason of arr...


In [15]:
fhlo_translated.to_csv('../../input_data/fully_translated_hand_labelled_output.csv')